In [4]:
# %pip install requests
# %pip install ssl
# %pip install parser-libraries
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.0 kB 187.9 kB/s eta 0:00:02
   ---- ---------------------------------- 30.7/250.0 kB 187.9 kB/s eta 0:00:02
   ------ -------------------------------- 41.0/250.0 kB 151.3 kB/s eta 0:00:02
   --------- ----------------------------- 61.4/250.0 kB 204.8 kB/s eta 0:00:01
   --------- ----------------------------- 61.4/250.0 kB 204.8 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.0 kB 249.8 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.0 kB 249.8 kB/s eta 0:00:01
   -------

In [19]:
import requests
from bs4 import BeautifulSoup
import openpyxl
from datetime import datetime

excel = openpyxl.Workbook()
sheet = excel.active
sheet.title = "Weather"
sheet.append(["Date","Day","High Tempertaure","Low Temperature","Conditions","Rain Percentage","Wind direction","Wind percentage"])

content = requests.get("https://weather.com/en-IN/weather/tenday/l/a5f0fe2ff9a40acc9ce62d67cd99439a71cde78cc0c5c1fbf6da052bef4cdba9")
soup = BeautifulSoup(content.text, 'html.parser')

weathers = soup.find_all('summary',class_= "Disclosure--Summary--3GiL4 DaypartDetails--Summary--3Fuya Disclosure--hideBorderOnSummaryOpen--3_ZkO Disclosure--tendayDisclosure--3cJgY")
for weather in weathers:
    date_str = weather.find('h3',class_= "DetailsSummary--daypartName--kbngc").text.split()[-2:]
    day_of_week = date_str[0]  # Extract day of the week
    day_of_month = date_str[1]  # Extract day of the month
    # Convert the month name to a datetime object to get the month number
    month_number = datetime.strptime(day_of_week, '%a').month
    # Format the date as 'dd/mm/yyyy'
    date = f"{day_of_month.zfill(2)}/{str(month_number).zfill(2)}/2024"
    day = weather.find('h3',class_= "DetailsSummary--daypartName--kbngc").text.split()[0]
    high_temp = int(weather.find('span', class_="DetailsSummary--highTempValue--3PjlX").text[0:2].replace('°', ''))
    low_temp = int(weather.find('span', class_="DetailsSummary--lowTempValue--2tesQ").text[0:2].replace('°', ''))
    conditions = weather.find('span', class_="DetailsSummary--extendedData--307Ax").text
    rain = int(weather.find('div',class_="DetailsSummary--precip--1a98O").text[-2:-1])
    wind_dir = weather.find('span',class_="Wind--windWrapper--3Ly7c undefined").text.split(' ')[0]
    wind_per = weather.find('span',class_="Wind--windWrapper--3Ly7c undefined").text.split(' ')[1]
    # print(date,day,high_temp,low_temp,conditions,rain,wind_dir,wind_per)

    sheet.append([date,day,high_temp,low_temp,conditions,rain,wind_dir,wind_per])

excel.save("weather.xlsx")
